# **BART MODEL TRAINING**

## **Prepare and split the extracted data for Training**

In [ ]:
!python "/content/discharge_summary_generation/dataset_creation/data_preparation.py"

In [ ]:
!python "/content/drive/MyDrive/discharge_summary_generation/discharge_summary_generation-main/dataset_creation/data_split.py"

In [ ]:
import pandas as pd
data = pd.read_csv("/content/mimic-iii_discharge_summary.csv")
data = pd.DataFrame(data)

In [ ]:
data.columns

Index(['Unnamed: 0', 'hadm_id', 'input_text', 'output_text'], dtype='object')

In [ ]:
is_colab = True

In [ ]:
if is_colab:
    !pip install datasets
    !pip install rouge_score
    !pip install transformers==4.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d139d78c88b134e2dc470d85fa2e44c8d02738ad6f73965fdb1914a4752305bc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 19.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requi

In [ ]:
!pip install datasets
!git clone https://github.com/Shaumik-Ashraf/BART-MIMIC-CXR.git
!pip install -r /home/azureuser/cloudfiles/code/Users/kaviyasampoornam/discharge_summary_generation-main/bart/transformers/seq2seq/requirements.txt

fatal: destination path 'BART-MIMIC-CXR' already exists and is not an empty directory.
     |████████████████████████████████| 118 kB 848 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 3.3 MB/s  eta 0:00:01
     |████████████████████████████████| 7.1 MB 70.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 44.0 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 68.7 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 4.9 MB/s  eta 0:00:01
     |████████████████████████████████| 139 kB 73.7 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 412 kB 75.6 MB/s eta 0:00:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24936 sha256=7a96999b247a93f1c94d0b906c8692c35d860c02e2bcda3f80cdc038be3962d5
  Stored in directory: /home/azureuser/.cache/pip/wheels/24/55/6f/eb

In [ ]:
import torch
import transformers
from transformers import BartModel, BartTokenizer, BartForConditionalGeneration, BartConfig
from transformers.models.bart.modeling_bart import shift_tokens_right
import csv
import numpy as np
import pandas as pd
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
directory = r'/home/azureuser/cloudfiles/code/Users/kaviyasampoornam/discharge_summary_generation-main'

## **Tokenize and preprocess data  to feed into model**

In [ ]:
def load_file(filename):
	print(f"Loading data from {filename}...");
	df = pd.read_csv(filename)
	print(f"Done.");
	return( np.array(df) );

def load_bart(model_name='facebook/bart-large-cnn', tokenizer_name='facebook/bart-large-cnn'):
	print(f"Loading pretrained model {model_name}...");
	model = bartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
	print("Done.");
	print(f"Loading pretrained tokenizer {tokenizer_name}...");
	tokenizer = bartTokenizer.from_pretrained('facebook/bart-large-cnn')
	print("Done.");
	return((model, tokenizer));

def basebart(article_to_summarize, model, tokenizer):
	inputs = tokenizer([article_to_summarize], max_length=1024, return_tensors='pt')
	inputs.to(device)
	summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=25, early_stopping=True)
	return [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

def write_csv_row(opened_file, row, model, tokenizer):
	comp_summary = basebart(row[2], model, tokenizer)
	opened_file.write(f"\"{row[0]}\",\"{row[1]}\",\"{comp_summary}\",\"{row[3]}\"\n");
	return(comp_summary);

In [ ]:
TEST_FILE = '/home/azureuser/cloudfiles/code/Users/kaviyasampoornam/discharge_summary_generation-main/test_small.csv'
LIMIT = -1
SUMMARIES_FILE = '/home/azureuser/cloudfiles/code/Users/kaviyasampoornam/discharge_summary_generation-main/' + str(LIMIT) + '.csv'

In [ ]:
print("==================== Start abstractive summarization ======================");

data = load_file(TEST_FILE);
model, tokenizer = load_bart();
model.to(device)

print(f"Writing {os.path.basename(SUMMARIES_FILE)}...");
f = open(SUMMARIES_FILE, 'w');
f.write(f"\"subject_id\",\"study_id\",\"prediction\",\"actual\"\n");
i = 0;
if LIMIT==-1: # based on the limit, print progress messages appropriately
	for row in data:
		write_csv_row(f, row, model, tokenizer);
		if( (i%1000 == 0) or (i+1 == LIMIT) ):
			print(f"Computed {i+1} summaries");
		i += 1;
elif LIMIT < 100:
	for row in data[:LIMIT]:
		write_csv_row(f, row, model, tokenizer);
		if( (i%(int(LIMIT/4)) == 0) or (i+1 == LIMIT)):
			print(f"Computed {i+1} summaries");
		i += 1;
else:
	for row in data[:LIMIT]:
		write_csv_row(f, row, model, tokenizer);
		if( (i%(int(LIMIT/8)) == 0) or (i+1 == LIMIT) ):
			print(f"Computed {i+1} summaries");
		i += 1;

f.close();
print("Done.\n");
print("==================== End abstractive summarization ======================");

In [ ]:
!python /home/azureuser/cloudfiles/code/Users/kaviyasampoornam/discharge_summary_generation-main/bart/transformers/seq2seq/run_summarization.py --model_name facebook/bart-large-cnn --tokenizer_name facebook/bart-large-cnn --output_dir output_small --train_file /home/azureuser/cloudfiles/code/Users/kaviyasampoornam/discharge_summary_generation-main/train_small.csv --validation_file /home/azureuser/cloudfiles/code/Users/kaviyasampoornam/discharge_summary_generation-main/val_small.csv --test_file /home/azureuser/cloudfiles/code/Users/kaviyasampoornam/discharge_summary_generation-main/test_small.csv --text_column input_text --summary_column output_text --max_source_length 1024 --max_target_length 25 --num_beams 4 --do_train --do_eval --do_predict --predict_with_generate

2023-08-02 06:14:53.803718: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 06:15:08.426711: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-02 06:15:08.426954: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-08-02 06:15:08.426974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

## **Save model to load and use**

In [ ]:
torch.save(model,"model.pth")

In [ ]:
model.save_pretrained("/content/drive/MyDrive/model/")
tokenizer.save_pretrained("/content/drive/MyDrive/model/")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_path = '/content/drive/MyDrive/model/'
tokenizer2 = AutoTokenizer.from_pretrained(model_path)
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_path)

### **Text Chunking**

In [ ]:
def long_text(input_sentence, max_length=512, max_new_tokens=32):
    # Split the input text into smaller segments with a specified maximum length
    device = "cpu"
    segments = [input_sentence[i:i + max_length] for i in range(0, len(input_sentence), max_length)]
    print(len(segments))
    # Generate for each segment separately
    generated_sentences = []
    for i, segment in enumerate(segments):
        batch = tokenizer2("paraphrase: " + segment if i > 0 else segment,
                          return_tensors='pt',
                          truncation=True,
                          padding=True,
                          max_length=max_length)
        batch = batch.to(device)
        generated_ids = model2.generate(batch.input_ids, max_new_tokens=max_new_tokens)
        generated_sentence = tokenizer2.batch_decode(generated_ids, skip_special_tokens=True)
        generated_sentences.append(generated_sentence[0])

    # Concatenate the generated text
    final_generated_sentence = " ".join(generated_sentences)
    return final_generated_sentence
import pandas as pd
data = pd.read_csv("/content/Book1.csv")
input_sentence = data.input[0]
result = long_text(input_sentence)
print(result)


19
A female with a history of emphysema, diagnosed by her primary care doctor and diagnosed with COPD flares, was admitted My daughter requires medication to maintain a higher oxygen saturation level than 90%. She has been on nebulizers and levofloxacin The condition can be attributed to om complaints, neurologic changes, rashes, palpitations, and orthopnea. It The elderly female described in the article had a symptomatic history of lightheadedness and headache, but the study revealed that Norvasc had a The CPAP machine makes it difficult to assess the exam, and there is no evidence of jugular venous pressure. The neck exam is The patient was admitted to the Medical Intensive Care Unit for COPD/dyspnea/pneumonia. She was given The patient underwent chest PT and had to use nebulizers for approximately 0.5 h (100 mL) due to persistent Dr. was given a rigid bronchoscopy with general anesthesia, where a stem of 12 x 25 and a portion of the The Interventional Pulmonology service examined th